In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
filepath = Path("../sample_files/lvtetmesh_coarse_fibers_LDRB_1.vtk") 
lv = LV.from_pyvista_read(filepath)

In [3]:
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.APEX_BASE_REGIONS, overwrite=False)
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.EPI_ENDO, overwrite=False)
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.SURFS, overwrite=False)
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.AM_SURFS, overwrite=False)
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.SURFS_DETAILED, overwrite=False)

In [4]:
s = lv.create_surface_oi_from_surface(LV_MESH_DATA.SURFS_DETAILED)

In [5]:
lv.set_aortic_info()
lv.set_mitral_info()

In [6]:
aortic_bc = lv.create_spring_rim_bc("AORTIC_BC", LV_SURFS.BORDER_AORTIC)
mitral_bc = lv.create_spring_rim_bc("MITRAL_BC", LV_SURFS.BORDER_MITRAL)

In [7]:
# lv.to_json("sample_lvhex_wfibers.json", 
#            nodeset_enums=[LV_SURFS],
#            surfaces_oi_enums=[LV_SURFS],
#            mesh_cell_data=[LV_FIBERS.F0.value, LV_FIBERS.S0.value, LV_FIBERS.N0.value])

In [8]:
N_NODES = lv.mesh.n_points
N_NODES_AORTIC = len(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"])

In [9]:
adjusted_discrete_sets = {}
to_adj = N_NODES
for key, values in lv._discrete_sets.items():
  adj_vals = np.copy(values) + 1
  adj_vals[:, 1] += to_adj
  adjusted_discrete_sets[key] = adj_vals
  to_adj += len(lv.get_bc(key)[1]["RIM_NODES"]) # account for nodes

In [10]:
adjusted_discrete_sets

{'AORTIC_BC': array([[   42, 64527],
        [   43, 64527],
        [   44, 64528],
        ...,
        [52674, 64529],
        [52675, 64559],
        [52678, 64555]], dtype=int64),
 'MITRAL_BC': array([[    1, 64658],
        [  131, 64701],
        [  132, 64701],
        ...,
        [52655, 64656],
        [52694, 64654],
        [52711, 64645]], dtype=int64)}

In [11]:
from pathlib import Path
from febio_python.feb import FEBio_feb
# ----------------------
# Read template file
feb_template_path = Path("../sample_files/sample_lv_template.feb")
feb = FEBio_feb.from_file(feb_template_path)
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Boundary: 1
-> Loads: 1
-> Discrete: 2
-> LoadData: 3
-> Output: 1
-> MeshData: 0

In [12]:
for bcname, vcvals in lv._bcs.items():
    break

In [13]:
vcvals

('RIM_SPRINGS',
 {'RIM_NODES': array([[-22.79303026,   3.64652812,  34.03378445],
         [-22.08817639,   4.05155757,  34.83948477],
         [-21.32755431,   4.35939693,  35.63742574],
         [-20.51848921,   4.56708153,  36.41992278],
         [-19.66877284,   4.67261124,  37.17944   ],
         [-18.78658844,   4.67496978,  37.90866283],
         [-17.88043191,   4.57413441,  38.60056847],
         [-16.95903005,   4.37107624,  39.24849348],
         [-16.03125646,   4.06775083,  39.846198  ],
         [-15.1060461 ,   3.66707938,  40.38792579],
         [-14.19230925,   3.17292055,  40.86845973],
         [-13.29884571,   2.59003339,  41.28317201],
         [-12.43426   ,   1.9240314 ,  41.62806872],
         [-11.60687858,   1.18132855,  41.89982833],
         [-10.82466956,   0.36907747,  42.09583363],
         [-10.09516606,  -0.50489941,  42.214197  ],
         [ -9.42539359,  -1.43218521,  42.25377851],
         [ -8.82180242,  -2.40384968,  42.214197  ],
         [ -8.290

In [14]:
# ----------------------
# Add Nodes
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  {"name": "AORTIC_RIM", 
   "nodes": np.round(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"], 4)
   },
  {"name": "MITRAL_RIM", 
   "nodes": np.round(lv.get_bc("MITRAL_BC")[1]["RIM_NODES"], 4)
   },
  ])

# ----------------------
# Add Elements
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    },
  {
    "name": "AORTIC_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": aortic_bc[LV_RIM.ELEMENTS.value] + N_NODES + 1 # adjust element ids
    },
  {
    "name": "MITRAL_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": mitral_bc[LV_RIM.ELEMENTS.value] + N_NODES + N_NODES_AORTIC + 1 # adjust element ids
    }
  ])

# ----------------------
# Add Nodesets
feb.add_nodesets(lv.get_nodesets_from_enum(LV_SURFS))

# ----------------------
# Add Surfaces
feb.add_surfaces({"ENDO": np.vstack(lv.get_surface_oi(LV_SURFS.ENDO)) + 1})

# ----------------------
# Add Discrete set
feb.add_discretesets(
  {"AORTIC_RIM_ELEMS": adjusted_discrete_sets["AORTIC_BC"],
   "MITRAL_RIM_ELEMS": adjusted_discrete_sets["MITRAL_BC"]}
  )

# add new 'discrete element to template
import xml.etree.ElementTree as ET
discrete = feb.discrete()
subel = ET.SubElement(discrete, "discrete")
subel.set("discrete_set","AORTIC_RIM_ELEMS")
subel.set("dmat","1")
subel = ET.SubElement(discrete, "discrete")
subel.set("discrete_set","MITRAL_RIM_ELEMS")
subel.set("dmat","1")

# ----------------------
# Add fibers
feb.add_meshdata([
  {
    "elem_set": "LV", 
    "var": "mat_axis",
    "elems": {
      "a": lv.get(GEO_DATA.MESH_CELL_DATA, LV_FIBERS.F0),
      "d": lv.get(GEO_DATA.MESH_CELL_DATA, LV_FIBERS.S0),
      }
    }
  ])

# ----------------------
# Check modified feb object
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 24
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Nodes 'MITRAL_RIM': 128
--> Elements 'LV': 218359
--> Elements 'AORTIC_RIM_ELEMS': 63
--> Elements 'MITRAL_RIM_ELEMS': 63
-> Boundary: 1
-> Loads: 1
-> Discrete: 3
-> LoadData: 3
-> Output: 1
-> MeshData: 1

In [15]:
# ----------------------
# Save
feb.write("../sample_files/sample_lv_template_mod.feb")

In [16]:
lv.to_feb_template(feb_template_path, 
    "../sample_files/sample_lv_template_mod_2.feb",
    log_level=10
    )

DEBUG:LV:Loading template: ..\sample_files\sample_lv_template.feb
DEBUG:LV:Checking for Nodes and Elements...
DEBUG:LV:No nodes found. Adding 'LV' nodes.
DEBUG:LV:No nodes found. Adding 'LV' elements.
DEBUG:LV:Checking for Nodesets...
DEBUG:LV:Checking for Surfaces...
DEBUG:LV:Adding surface_oi  'ENDO'
DEBUG:LV:Adding surface_oi  'EPI'
DEBUG:LV:Adding surface_oi  'ENDO_AORTIC'
DEBUG:LV:Adding surface_oi  'EPI_AORTIC'
DEBUG:LV:Adding surface_oi  'BORDER_AORTIC'
DEBUG:LV:Adding surface_oi  'ENDO_MITRAL'
DEBUG:LV:Adding surface_oi  'EPI_MITRAL'
DEBUG:LV:Adding surface_oi  'BORDER_MITRAL'
DEBUG:LV:Adding surface_oi  'ENDO_AM_INTERCECTION'
DEBUG:LV:Adding surface_oi  'EPI_AM_INTERCECTION'
DEBUG:LV:Checking for Boundary conditions...
DEBUG:LV:Adding BC  'AORTIC_BC'
DEBUG:LV:Adding BC  'MITRAL_BC'
DEBUG:LV:Checking for dicrete sets...
DEBUG:LV:Discrete set to add: 'AORTIC_BC'
DEBUG:LV:Discrete set to add: 'MITRAL_BC'
DEBUG:LV:Checking for fibers...
DEBUG:LV:Fiber data successfully added.
DEBU